In [1]:
import os,glob
import pandas as pd
import numpy as np
import re

change country between: PR, DR, CU

In [2]:
user = "Chris"
country = "PR"

In [3]:
CU_directory = r"/Users/chrissoria/Google Drive/Other Computers/My Laptop (1)/documents/cadas/data/CADAS data upload/Cuba"
DR_directory = r"/Users/chrissoria/Google Drive/Other Computers/My Laptop (1)/documents/cadas/data/CADAS data upload/Rep Dom"
PR_directory = r"/Users/chrissoria/Google Drive/Other Computers/My Laptop (1)/documents/cadas/data/CADAS data upload/Puerto_Rico"

if user == "Chris":
    if country == "DR":
        current_directory = DR_directory
        path1 = DR_directory+"/All_CSVs" #where all CSVs are located
        path2 = DR_directory+"/Sync Files/2024_10_12/CSV/" #change to show latest upload
        imagepath = DR_directory+"/All_Image_Files"
        audiopath = DR_directory+"/All_Audio_Files"
    elif country == "CU":
        current_directory = CU_directory
        path1 = CU_directory+"/All_CSVs" #where we merge all cumulative CSV files into one file
        imagepath = CU_directory+"/All_Image_Files"
        audiopath = CU_directory+"/All_Audio_Files"
        path2 = CU_directory+"/Sync_Files/09_06_24/CSV" #where we check to see that the latest batch of data includes all files it should
    elif country == "PR":
        current_directory = PR_directory
        path1 = PR_directory+"/All_CSVs"
        path2 = PR_directory+"/Sync_Files/2024_11_13/CSV/"
        imagepath = PR_directory+"/All_Image_Files"
        audiopath = PR_directory+"/All_Audio_Files"
elif user == "Ty":
    current_directory = r"C:\Users\Ty\Desktop\DataExport"
else:
    print("Unknown user. Cannot set directory.")
    current_directory = None

if current_directory:
    os.chdir(current_directory)
    cropped_images_path= "/Users/chrissoria/Google Drive/Other Computers/My Laptop (1)/documents/cadas/data/CADAS data upload/Pentagons_for_consensus_website"
    file_list = os.listdir(current_directory)
    print(f"Changed directory to: {current_directory}")
    print(cropped_images_path)
    print(path1)
    print(path2)
    print(file_list)
    
imagenames = os.listdir(imagepath)
audionames = os.listdir(audiopath)

Changed directory to: /Users/chrissoria/Google Drive/Other Computers/My Laptop (1)/documents/cadas/data/CADAS data upload/Puerto_Rico
/Users/chrissoria/Google Drive/Other Computers/My Laptop (1)/documents/cadas/data/CADAS data upload/Pentagons_for_consensus_website
/Users/chrissoria/Google Drive/Other Computers/My Laptop (1)/documents/cadas/data/CADAS data upload/Puerto_Rico/All_CSVs
/Users/chrissoria/Google Drive/Other Computers/My Laptop (1)/documents/cadas/data/CADAS data upload/Puerto_Rico/Sync_Files/2024_11_13/CSV/
['CADAS Tracking Sheet.gsheet', 'CSV_Merged', '.DS_Store', 'credentiales de nuve.gsheet', 'All_Audio_Files', 'Resumen', 'Figures', 'All_Image_Files', 'Sync_files', 'All_CSVs']


here, we're checking to see if there's the same amount of objects in he incoming sync files compared to after they're processed.

In [4]:
incoming_SYNC = os.listdir(path2+'../SYNC')
incoming_CSV = os.listdir(path2)

incoming_SYNC = [s.split('.')[0] for s in incoming_SYNC]
incoming_CSV = [s.split('.')[0] for s in incoming_CSV]

if len(incoming_SYNC) != len(incoming_CSV):
    print(f"Discrepancy in number of CSV compared to sync files. There are {len(incoming_SYNC)} in sync but {len(incoming_CSV)} in CSV.")
else:
    print("Correct number of CSV compared to sync files.")
    
for sync_file in incoming_SYNC:
    if sync_file not in incoming_CSV:
        print(f"File {sync_file} is not present in incoming_CSV")

Discrepancy in number of CSV compared to sync files. There are 354 in sync but 336 in CSV.
File Cognitiva_202411130935_0_PR03 is not present in incoming_CSV
File Sample_Contact_Investigation_202411080853_0_PR07 is not present in incoming_CSV
File Sample_Contact_Investigation_202303170824_0_PR07 is not present in incoming_CSV
File _informant_202411130935_0_PR03 is not present in incoming_CSV
File _cognitive_202404030233_0_PR06 is not present in incoming_CSV
File Sample_Contact_Investigation_202411070226_0_PR07 is not present in incoming_CSV
File Sample_Contact_Investigation_202411080148_0_PR04 is not present in incoming_CSV
File _informationdoor_202411130935_0_PR03 is not present in incoming_CSV
File Sample_Contact_Investigation_202408080501_0_PR04 is not present in incoming_CSV
File Sample_Contact_Investigation_202407110055_0_PR04 is not present in incoming_CSV
File Sample_Contact_Investigation_202406060900_0_PR10 is not present in incoming_CSV
File Sample_Contact_Investigation_2024110

In [5]:
cropped_image_list = os.listdir(cropped_images_path)
cropped_image_list = [image for image in cropped_image_list if "(" not in image]

Received files document

In [6]:
# Get the list of all files in the latest file upload
dir_list1 = os.listdir(path2)

# To see which files I received from the data manager (for now not doing anything with this)
newdir_list1 = [x for x in dir_list1 if x.endswith('.epi7')]

# To see which files I was able to process with Epi Info tool
dir_list2 = os.listdir(path2)
newdir_list2 = [x for x in dir_list2 if x.endswith('.csv')]

def remove_common(a, b):
    for i in a[:]:
        if i in b:
            a.remove(i)
            b.remove(i)

    print("list1 : ", a)
    print("list2 : ", b)

remove_common(newdir_list1, newdir_list2)

words_to_filter = ["Neighborhood", "Door", "_informationdoor_", "_informationdoorparticipants", "Listas", "_mainhousehold", "_participants", "_rosterhousehold", "_nonresidentchildren", "Cognitva", "_cognitive", "Cognitive_Scoring", "Examen_Físico", "_physical_exam","Familiar", "_household", "Informante", "_informant"]

# breaking the longer list into smaller lists
filtered_lists = {word: [] for word in words_to_filter}

for text in newdir_list2:
    for word in words_to_filter:
        if word in text:
            filtered_lists[word].append(text)

# change this number depending on how many tablets worth of information you're receiving
for word, filtered_list in filtered_lists.items():
    while len(filtered_list) < 27:
        filtered_list.append("")

Received_Files = pd.DataFrame()

for word, filtered_list in filtered_lists.items():
    Received_Files[word] = pd.Series(filtered_list)

Received_Files.to_excel(path2+'/Received_Files_Checklist.xlsx', index=False)

Received_Files

list1 :  []
list2 :  ['_physical_exam_202408210257_0_PR06.csv', '_physical_exam_202411080211_0_PR06.csv', '_informationdoorparticipants_202411080210_0_PR06.csv', 'Examen_Físico_202411130934_0_PR10.csv', 'Sample_Contact_Investigation_202403220023_0_PR06.csv', '_informant_202411070219_0_PR07.csv', 'Cognitive_Scoring_202303170823_0_PR07.csv', 'Cognitive_Scoring_202411211025_0_PR06.csv', 'Examen_Físico_202303230152_0_d7d6f37d44ac4011.csv', '_physical_exam_202411081105_0_PR07.csv', '_informant_202407110055_0_PR04.csv', '_mainhousehold_202411081105_0_PR07.csv', '_household_202411080854_0_PR07.csv', 'Cognitive_Scoring_202411080852_0_PR07.csv', 'Sociodemográfica_202411070226_0_PR07.csv', 'Cognitive_Scoring_202411130935_0_PR03.csv', 'Sociodemográfica_202411130935_0_PR03.csv', '_physical_exam_202408080501_0_PR04.csv', 'Cognitive_Scoring_202411070226_0_PR07.csv', '_mainhousehold_202411080211_0_PR06.csv', '_informationdoor_202411070225_0_PR07.csv', 'Listas_202411081142_0_PR01.csv', 'Informante

,Neighborhood,Door,_informationdoor_,_informationdoorparticipants,Listas,_mainhousehold,_participants,_rosterhousehold,_nonresidentchildren,Cognitva,_cognitive,Cognitive_Scoring,Examen_Físico,_physical_exam,Familiar,_household,Informante,_informant
0,Neighborhood_202411070219_0_PR07.csv,Door_202411081142_0_PR01.csv,_informationdoor_202411070225_0_PR07.csv,_informationdoorparticipants_202411080210_0_PR...,Listas_202411081142_0_PR01.csv,_mainhousehold_202411081105_0_PR07.csv,_participants_202411080854_0_PR07.csv,_rosterhousehold_202411080854_0_PR07.csv,_nonresidentchildren_202411130935_0_PR10.csv,,_cognitive_202411080148_0_PR04.csv,Cognitive_Scoring_202303170823_0_PR07.csv,Examen_Físico_202411130934_0_PR10.csv,_physical_exam_202408210257_0_PR06.csv,Familiar_202411070218_0_PR07.csv,_household_202411080854_0_PR07.csv,Informante_202411080147_0_PR04.csv,_informant_202411070219_0_PR07.csv
1,Neighborhood_202411080211_0_PR06.csv,Door_202408280102_0_PR10.csv,_informationdoor_202411080854_0_PR07.csv,_informationdoorparticipants_202411211024_0_PR...,Listas_202411080854_0_PR07.csv,_mainhousehold_202411080211_0_PR06.csv,_participants_202411080209_0_PR06.csv,_rosterhousehold_202411081105_0_PR07.csv,_nonresidentchildren_202411081141_0_PR01.csv,,_cognitive_202303170824_0_PR07.csv,Cognitive_Scoring_202411211025_0_PR06.csv,Examen_Físico_202303230152_0_d7d6f37d44ac4011...,_physical_exam_202411080211_0_PR06.csv,Familiar_202411130934_0_PR10.csv,_household_202411081142_0_PR01.csv,Informante_202411070219_0_PR07.csv,_informant_202407110055_0_PR04.csv
2,Neighborhood_202411130935_0_PR03.csv,Door_202411080209_0_PR06.csv,_informationdoor_202411211024_0_PR06.csv,_informationdoorparticipants_202411070218_0_PR...,Listas_202411080209_0_PR06.csv,_mainhousehold_202303170823_0_PR07.csv,_participants_202411211024_0_PR06.csv,_rosterhousehold_202411130935_0_PR03.csv,_nonresidentchildren_202411070219_0_PR07.csv,,_cognitive_202408210258_0_PR06.csv,Cognitive_Scoring_202411080852_0_PR07.csv,Examen_Físico_202303230555_0_d7d6f37d44ac4011...,_physical_exam_202411081105_0_PR07.csv,Familiar_202411080210_0_PR06.csv,_household_202411081106_0_PR07.csv,Informante_202407110055_0_PR04.csv,_informant_202411070226_0_PR07.csv
3,Neighborhood_202411070226_0_PR07.csv,Door_202411130934_0_PR10.csv,_informationdoor_202411080210_0_PR06.csv,_informationdoorparticipants_202411080209_0_PR...,Listas_202411130934_0_PR10.csv,_mainhousehold_202411080852_0_PR07.csv,_participants_202411080210_0_PR06.csv,_rosterhousehold_202411070226_0_PR07.csv,_nonresidentchildren_202411081105_0_PR07.csv,,_cognitive_202407110055_0_PR04.csv,Cognitive_Scoring_202411130935_0_PR03.csv,Examen_Físico_202411211024_0_PR06.csv,_physical_exam_202408080501_0_PR04.csv,Familiar_202408080500_0_PR04.csv,_household_202303230656_0_d7d6f37d44ac4011.csv,Informante_202411081105_0_PR07.csv,_informant_202411080211_0_PR06.csv
4,Neighborhood_202411211025_0_PR06.csv,Door_202411080210_0_PR06.csv,_informationdoor_202411070218_0_PR07.csv,_informationdoorparticipants_202411070225_0_PR...,Listas_202411211024_0_PR06.csv,_mainhousehold_202411070226_0_PR07.csv,_participants_202408210258_0_PR06.csv,_rosterhousehold_202411080852_0_PR07.csv,_nonresidentchildren_202303170823_0_PR07.csv,,_cognitive_202411070219_0_PR07.csv,Cognitive_Scoring_202411070226_0_PR07.csv,Examen_Físico_202406050350_0_PR04.csv,_physical_exam_202411130935_0_PR03.csv,Familiar_202411211024_0_PR06.csv,_household_202303230657_0_d7d6f37d44ac4011.csv,Informante_202408210257_0_PR06.csv,_informant_202408080501_0_PR04.csv
5,Neighborhood_202303170824_0_PR07.csv,Door_202411211024_0_PR06.csv,_informationdoor_202411080209_0_PR06.csv,_informationdoorparticipants_202411080854_0_PR...,Listas_202411080853_0_PR07.csv,_mainhousehold_202411130935_0_PR03.csv,_participants_202411080148_0_PR04.csv,_rosterhousehold_202303170823_0_PR07.csv,_nonresidentchildren_202411080852_0_PR07.csv,,_cognitive_202411211025_0_PR06.csv,Cognitive_Scoring_202411081105_0_PR07.csv,Examen_Físico_202411080853_0_PR07.csv

In [7]:
if user == "Chris":
    if country == "CU":
        pattern = "/Users/chrissoria/Google Drive/Other Computers/My Laptop \(1\)/documents/cadas/data/CADAS data upload/Cuba/All_CSVs/"
    elif country == "DR":
        pattern = "/Users/chrissoria/Google Drive/Other Computers/My Laptop \(1\)/documents/cadas/data/CADAS data upload/Rep \Dom/All_CSVs/"
    elif country == "PR":
        pattern = "/Users/chrissoria/Google Drive/Other Computers/My Laptop \(1\)/documents/cadas/data/CADAS data upload/Puerto_Rico/All_CSVs/"
elif user == "Ty":
    if country == "CU":
        pattern = r"C:\Users\Ty\Desktop\DataExport/"
    elif country == "DR":
        pattern = r"C:\Users\Ty\Desktop\DataExport/"
else:
    print("Unknown user. Cannot set directory.")
    current_directory = None

folder_path = path1 #goes through this whole folder

####COGNITIVE CHILD#######

dfs = []

for filename in glob.glob(os.path.join(folder_path, '*_cognitive*.csv')):

    print(f"Processing {filename}")
    df = pd.read_csv(filename)
    df['filename'] = filename
    pattern_to_remove = pattern + "_cognitive_"
    df['filename'] = df['filename'].str.replace(pattern_to_remove, '', regex=True)
    df['filename'] = df['filename'].str.replace(r'_.*', '', regex=True)
    df['filename'] = df['filename'].astype(int)
    dfs.append(df)
    
cog_child = pd.concat(dfs, ignore_index=True)

cog_child['num_missing'] = cog_child.isna().sum(axis=1)

# Adjust 'num_missing' so that rows with fewer missing values have higher numbers
cog_child['num_missing'] = cog_child['num_missing'].max() - cog_child['num_missing']

# Sort the DataFrame by 'num_missing' in descending order
cog_child = cog_child.sort_values(by='num_missing', ascending=False)

# Drop the 'filename' column
cog_child = cog_child.drop(columns=['filename'])

#restoring old images
images = [
    'C_72_1_PIC', 'C_72_2_PIC', 'C_72_3_PIC', 'C_72_4_PIC',
    'C_79_1_PIC', 'C_79_2_PIC', 'C_79_3_PIC', 'C_79_4_PIC',
    'ANIM_PIC', 'SYMB_PIC', 'PENT_PIC', 'GlobalRecordId'
]
images_df = cog_child[images]

columns = images_df.columns
rename_dict = {col: f"{col}_restored" for col in columns[:-1]}
images_df.rename(columns=rename_dict, inplace=True)

images_df = images_df.sort_values(by=['GlobalRecordId', 'PENT_PIC_restored'], ascending=[True, False], na_position='last')
images_df = images_df.drop_duplicates(subset='GlobalRecordId', keep='first')

cog_child = cog_child.drop_duplicates(subset='GlobalRecordId', keep='first')

cog_child = cog_child.merge(images_df, on='GlobalRecordId', how='left')

for col, restored_col in rename_dict.items():
    cog_child[col] = cog_child.apply(
        lambda row: row[restored_col] if pd.isna(row[col]) else row[col],
        axis=1
    )
    
cog_child = cog_child.drop(columns=cog_child.filter(like='_restored').columns)

#done restoring images here

pattern_to_remove = r'=HYPERLINK\("media\\([^"]+)",\"<CLICK HERE>"\)'
cog_child = cog_child.replace(pattern_to_remove, r'\1', regex=True)

if country == "CU":
    cog_child['C_72_1_PIC'] = np.nan
    cog_child['C_72_2_PIC'] = np.nan
    cog_child['C_72_3_PIC'] = np.nan
    cog_child['C_79_1_PIC'] = np.nan
    cog_child['C_79_2_PIC'] = np.nan
    cog_child['C_79_3_PIC'] = np.nan

columns_to_check = [
    'C_72_1_PIC', 'C_72_2_PIC', 'C_72_3_PIC', 'C_72_4_PIC',
    'C_79_1_PIC', 'C_79_2_PIC', 'C_79_3_PIC', 'C_79_4_PIC',
    'ANIM_PIC', 'SYMB_PIC', 'PENT_PIC',
    'G_1_File', 'G_2_File', 'G_2_File2', 'G_3_File', 'G_3_File2'
]

for column in columns_to_check:
    foundlist = []
    piclist = cog_child[column].tolist()
    file_list = audionames if 'File' in column else imagenames

    for pic in piclist:
        if pic in file_list:
            foundlist.append("found")
        else:
            foundlist.append("missing")

    cog_child[column + '_FOUND'] = foundlist
    
cog_child['pent_pic_cropped'] = cog_child['PENT_PIC'].apply(lambda x: 'cropped' if x in cropped_image_list else pd.NA)

for column in cog_child.columns:
    if "TIME" in column:
        # Apply transformation to each cell in the column
        cog_child[column] = cog_child[column].apply(lambda x: re.sub(r'(.*?:.*?):..', r'\1', x) if pd.notnull(x) else x)
        
cog_child.to_csv("CSV_Merged/Cog_Child.csv")

####COGNITIVE PARENT#######

dfs = []

for filename in glob.glob(os.path.join(folder_path, '*Cognitiva*.csv')):

    print(f"Processing {filename}")
    df = pd.read_csv(filename)
    df['filename'] = filename
    pattern_to_remove = pattern + 'Cognitiva_'
    df['filename'] = df['filename'].str.replace(pattern_to_remove, '', regex=True)
    df['filename'] = df['filename'].str.replace(r'_.*', '', regex=True)
    df['filename'] = df['filename'].astype(int)
    dfs.append(df)
    
cog_parent = pd.concat(dfs, ignore_index=True)
cog_parent = cog_parent.sort_values(by='filename', ascending=False)
cog_parent = cog_parent.drop(columns=['filename'])
cog_parent = cog_parent.drop_duplicates(subset='GlobalRecordId', keep='first')

cog_parent.to_csv("CSV_Merged/Cog_Parent.csv")

####COGNITIVE SCORING#######

dfs = []

for filename in glob.glob(os.path.join(folder_path, '*Cognitive_Scoring*.csv')):

    print(f"Processing {filename}")
    df = pd.read_csv(filename)
    df['filename'] = filename
    pattern_to_remove = pattern + 'Cognitive_Scoring_'
    df['filename'] = df['filename'].str.replace(pattern_to_remove, '', regex=True)
    df['filename'] = df['filename'].str.replace(r'_.*', '', regex=True)
    df['filename'] = df['filename'].astype(int)
    dfs.append(df)
    
cog_scoring = pd.concat(dfs, ignore_index=True)
cog_scoring['num_missing'] = cog_scoring.isna().sum(axis=1)
cog_scoring['num_missing'] = cog_scoring['num_missing'].max() - cog_scoring['num_missing']

cog_scoring = cog_scoring.sort_values(by='filename', ascending=False)

cog_scoring = cog_scoring.drop(columns=['filename','num_missing'])

cog_scoring = cog_scoring.drop_duplicates(subset='GlobalRecordId', keep='first')

for column in cog_scoring.columns:
    if "TIME" in column:
        # Apply transformation to each cell in the column
        cog_scoring[column] = cog_scoring[column].apply(lambda x: re.sub(r'(.*?:.*?):..', r'\1', x) if pd.notnull(x) else x)

cog_scoring.to_csv("CSV_Merged/Cog_Scoring.csv")

####HOUSEHOLD CHILD#######

dfs = []

for filename in glob.glob(os.path.join(folder_path, '*_household*.csv')):

    print(f"Processing {filename}")
    df = pd.read_csv(filename)
    df['filename'] = filename
    pattern_to_remove = pattern + '_household_'
    df['filename'] = df['filename'].str.replace(pattern_to_remove, '', regex=True)
    df['filename'] = df['filename'].str.replace(r'_.*', '', regex=True)
    df['filename'] = df['filename'].astype(int)
    dfs.append(df)
    
household_child = pd.concat(dfs, ignore_index=True)
household_child['num_missing'] = household_child.isna().sum(axis=1)
household_child['num_missing'] = household_child['num_missing'].max() - household_child['num_missing']

household_child = household_child.sort_values(by='filename', ascending=False)

household_child = household_child.drop_duplicates(subset='GlobalRecordId', keep='first')

household_child = household_child.drop(columns=['filename','num_missing'])


for column in household_child.columns:
    if "TIME" in column:
        # Apply transformation to each cell in the column
        household_child[column] = household_child[column].apply(lambda x: re.sub(r'(.*?:.*?):..', r'\1', x) if pd.notnull(x) else x)


household_child.to_csv("CSV_Merged/Household_Child.csv")

####HOUSEHOLD PARENT#######

dfs = []

for filename in glob.glob(os.path.join(folder_path, '*Familiar*.csv')):

    print(f"Processing {filename}")
    df = pd.read_csv(filename)
    df['filename'] = filename
    pattern_to_remove = pattern + 'Familiar_'
    df['filename'] = df['filename'].str.replace(pattern_to_remove, '', regex=True)
    df['filename'] = df['filename'].str.replace(r'_.*', '', regex=True)
    df['filename'] = df['filename'].astype(int)
    dfs.append(df)
    
household_parent = pd.concat(dfs, ignore_index=True)
household_parent = household_parent.sort_values(by='filename', ascending=False)
household_parent = household_parent.drop(columns=['filename'])
household_parent = household_parent.drop_duplicates(subset='GlobalRecordId', keep='first')

household_parent.to_csv("CSV_Merged/Household_Parent.csv")

####PHYS CHILD#######

dfs = []

for filename in glob.glob(os.path.join(folder_path, '*_physical_exam*.csv')):

    print(f"Processing {filename}")
    df = pd.read_csv(filename)
    df['filename'] = filename
    pattern_to_remove = pattern + '_physical_exam_'
    df['filename'] = df['filename'].str.replace(pattern_to_remove, '', regex=True)
    df['filename'] = df['filename'].str.replace(r'_.*', '', regex=True)
    df['filename'] = df['filename'].astype(int)
    dfs.append(df)
    
phys_child = pd.concat(dfs, ignore_index=True)
phys_child['num_missing'] = phys_child.isna().sum(axis=1)
phys_child['num_missing'] = phys_child['num_missing'].max() - phys_child['num_missing']

phys_child = phys_child.sort_values(by=['filename', 'num_missing'], ascending=[False, True])

phys_child = phys_child.drop(columns=['filename','num_missing'])

phys_child = phys_child.drop_duplicates(subset='GlobalRecordId', keep='first')

for column in phys_child.columns:
    if "TIME" in column:
        # Apply transformation to each cell in the column
        phys_child[column] = phys_child[column].apply(lambda x: re.sub(r'(.*?:.*?):..', r'\1', x) if pd.notnull(x) else x)


phys_child.to_csv("CSV_Merged/Phys_Child.csv")

####PHYS PARENT#######

dfs = []

for filename in glob.glob(os.path.join(folder_path, '*Examen_Físico*.csv')):

    print(f"Processing {filename}")
    df = pd.read_csv(filename)
    df['filename'] = filename
    pattern_to_remove = pattern + 'Examen_Físico_'
    df['filename'] = df['filename'].str.replace(pattern_to_remove, '', regex=True)
    df['filename'] = df['filename'].str.replace(r'_.*', '', regex=True)
    df['filename'] = df['filename'].astype(int)
    dfs.append(df)
    
phys_parent = pd.concat(dfs, ignore_index=True)
phys_parent = phys_parent.sort_values(by='filename', ascending=False)
phys_parent = phys_parent.drop(columns=['filename'])
phys_parent = phys_parent.drop_duplicates(subset='GlobalRecordId', keep='first')

phys_parent.to_csv("CSV_Merged/Phys_Parent.csv")

####INFOR CHILD#######

dfs = []

for filename in glob.glob(os.path.join(folder_path, '*_informant*.csv')):

    print(f"Processing {filename}")
    df = pd.read_csv(filename)
    df['filename'] = filename
    pattern_to_remove = pattern + '_informant_'
    df['filename'] = df['filename'].str.replace(pattern_to_remove, '', regex=True)
    df['filename'] = df['filename'].str.replace(r'_.*', '', regex=True)
    df['filename'] = df['filename'].astype(int)
    dfs.append(df)
    
infor_child = pd.concat(dfs, ignore_index=True)
infor_child['num_missing'] = infor_child.isna().sum(axis=1)
infor_child['num_missing'] = infor_child['num_missing'].max() - infor_child['num_missing']

infor_child = infor_child.sort_values(by=['filename', 'num_missing'], ascending=[False, True])

infor_child = infor_child.drop(columns=['filename','num_missing'])
infor_child = infor_child.drop_duplicates(subset='GlobalRecordId', keep='first')

for column in infor_child.columns:
    if "TIME" in column:
        # Apply transformation to each cell in the column
        infor_child[column] = infor_child[column].apply(lambda x: re.sub(r'(.*?:.*?):..', r'\1', x) if pd.notnull(x) else x)


infor_child.to_csv("CSV_Merged/Infor_Child.csv")

####INFOR PARENT#######

dfs = []

for filename in glob.glob(os.path.join(folder_path, '*Informante*.csv')):

    print(f"Processing {filename}")
    df = pd.read_csv(filename)
    df['filename'] = filename
    pattern_to_remove = pattern + 'Informante_'
    df['filename'] = df['filename'].str.replace(pattern_to_remove, '', regex=True)
    df['filename'] = df['filename'].str.replace(r'_.*', '', regex=True)
    df['filename'] = df['filename'].astype(int)
    dfs.append(df)
    
infor_parent = pd.concat(dfs, ignore_index=True)
infor_parent = infor_parent.sort_values(by='filename', ascending=False)
infor_parent = infor_parent.drop(columns=['filename'])
infor_parent = infor_parent.drop_duplicates(subset='GlobalRecordId', keep='first')

infor_parent.to_csv("CSV_Merged/Infor_Parent.csv")

####SOCIO CHILD#######

dfs = []

for filename in glob.glob(os.path.join(folder_path, '*_sociodemographic*.csv')):

    print(f"Processing {filename}")
    df = pd.read_csv(filename)
    df['filename'] = filename
    pattern_to_remove = pattern + '_sociodemographic_'
    df['filename'] = df['filename'].str.replace(pattern_to_remove, '', regex=True)
    df['filename'] = df['filename'].str.replace(r'_.*', '', regex=True)
    df['filename'] = df['filename'].astype(int)
    dfs.append(df)
    
socio_child = pd.concat(dfs, ignore_index=True)
socio_child['num_missing'] = socio_child.isna().sum(axis=1)
socio_child['num_missing'] = socio_child['num_missing'].max() - socio_child['num_missing']

socio_child = socio_child.sort_values(by=['filename', 'num_missing'], ascending=[False, True])

socio_child = socio_child.drop(columns=['filename','num_missing'])
socio_child = socio_child.drop_duplicates(subset='GlobalRecordId', keep='first')

for column in socio_child.columns:
    if "TIME" in column:
        # Apply transformation to each cell in the column
        socio_child[column] = socio_child[column].apply(lambda x: re.sub(r'(.*?:.*?):..', r'\1', x) if pd.notnull(x) else x)


socio_child.to_csv("CSV_Merged/Socio_Child.csv")

####SOCIO PARENT#######

dfs = []

for filename in glob.glob(os.path.join(folder_path, '*Sociodemográfica*.csv')):

    print(f"Processing {filename}")
    df = pd.read_csv(filename)
    df['filename'] = filename
    pattern_to_remove = pattern + 'Sociodemográfica_'
    df['filename'] = df['filename'].str.replace(pattern_to_remove, '', regex=True)
    df['filename'] = df['filename'].str.replace(r'_.*', '', regex=True)
    df['filename'] = df['filename'].astype(int)
    dfs.append(df)
    
socio_parent = pd.concat(dfs, ignore_index=True)
socio_parent = socio_parent.sort_values(by='filename', ascending=False)
socio_parent = socio_parent.drop(columns=['filename'])
socio_parent = socio_parent.drop_duplicates(subset='GlobalRecordId', keep='first')

socio_parent.to_csv("CSV_Merged/Socio_Parent.csv")

####NEIGHBORHOOD#######

dfs = []

for filename in glob.glob(os.path.join(folder_path, '*Neighborhood*.csv')):

    print(f"Processing {filename}")
    df = pd.read_csv(filename)
    df['filename'] = filename
    pattern_to_remove = pattern + 'Neighborhood_'
    df['filename'] = df['filename'].str.replace(pattern_to_remove, '', regex=True)
    df['filename'] = df['filename'].str.replace(r'_.*', '', regex=True)
    df['filename'] = df['filename'].astype(int)
    dfs.append(df)
    
neighborhood = pd.concat(dfs, ignore_index=True)
neighborhood['num_missing'] = neighborhood.isna().sum(axis=1)
neighborhood['num_missing'] = neighborhood['num_missing'].max() - neighborhood['num_missing']

neighborhood = neighborhood.sort_values(by=['filename', 'num_missing'], ascending=[False, True])

neighborhood = neighborhood.drop(columns=['filename','num_missing'])
neighborhood = neighborhood.drop_duplicates(subset='GlobalRecordId', keep='first')

neighborhood.to_csv("CSV_Merged/Neighborhood.csv")

####DOOR PARENT#######

dfs = []

for filename in glob.glob(os.path.join(folder_path, '*Door*.csv')):

    print(f"Processing {filename}")
    df = pd.read_csv(filename)
    df['filename'] = filename
    pattern_to_remove = pattern + 'Door_'
    df['filename'] = df['filename'].str.replace(pattern_to_remove, '', regex=True)
    df['filename'] = df['filename'].str.replace(r'_.*', '', regex=True)
    df['filename'] = df['filename'].astype(int)
    dfs.append(df)
    
door_parent = pd.concat(dfs, ignore_index=True)
door_parent = door_parent.sort_values(by='filename', ascending=False)
door_parent = door_parent.drop(columns=['filename'])
door_parent = door_parent.drop_duplicates(subset='GlobalRecordId', keep='first')

door_parent.to_csv("CSV_Merged/Door.csv")

####DOOR CHILD 1#######

dfs = []

for filename in glob.glob(os.path.join(folder_path, '*_informationdoor_*.csv')):

    print(f"Processing {filename}")
    df = pd.read_csv(filename)
    df['filename'] = filename
    pattern_to_remove = pattern + '_informationdoor_'
    df['filename'] = df['filename'].str.replace(pattern_to_remove, '', regex=True)
    df['filename'] = df['filename'].str.replace(r'_.*', '', regex=True)
    df['filename'] = df['filename'].astype(int)
    dfs.append(df)
    
informationdoor = pd.concat(dfs, ignore_index=True)

informationdoor['D_HOUSEID'] = informationdoor.groupby('GlobalRecordId')['D_HOUSEID'].transform(lambda x: x.ffill().bfill())
informationdoor['D_CLUSTID'] = informationdoor.groupby('GlobalRecordId')['D_CLUSTID'].transform(lambda x: x.ffill().bfill())
informationdoor['D_INTERID'] = informationdoor.groupby('GlobalRecordId')['D_INTERID'].transform(lambda x: x.ffill().bfill())

informationdoor['num_missing'] = informationdoor.isna().sum(axis=1)
informationdoor['num_missing'] = informationdoor['num_missing'].max() - informationdoor['num_missing']

informationdoor = informationdoor.sort_values(by=['filename', 'num_missing'], ascending=[False, True])

informationdoor = informationdoor.drop(columns=['filename','num_missing'])

informationdoor = informationdoor.drop_duplicates(subset='GlobalRecordId', keep='first')

informationdoor.to_csv("CSV_Merged/InformationDoor.csv")

####DOOR CHILD 2#######

dfs = []

for filename in glob.glob(os.path.join(folder_path, '*_informationdoorparticipants*.csv')):

    print(f"Processing {filename}")
    df = pd.read_csv(filename)
    df['filename'] = filename
    pattern_to_remove = pattern + '_informationdoorparticipants_'
    df['filename'] = df['filename'].str.replace(pattern_to_remove, '', regex=True)
    df['filename'] = df['filename'].str.replace(r'_.*', '', regex=True)
    df['filename'] = df['filename'].astype(int)
    dfs.append(df)
    
informationdoorparticipants = pd.concat(dfs, ignore_index=True)

informationdoorparticipants['num_missing'] = informationdoorparticipants.isna().sum(axis=1)
informationdoorparticipants['num_missing'] = informationdoorparticipants['num_missing'].max() - informationdoorparticipants['num_missing']

informationdoorparticipants = informationdoorparticipants.sort_values(by=['filename', 'num_missing'], ascending=[False, True])

informationdoorparticipants = informationdoorparticipants.drop(columns=['filename','num_missing'])

informationdoorparticipants = informationdoorparticipants.drop_duplicates(subset='GlobalRecordId', keep='first')

informationdoorparticipants.to_csv("CSV_Merged/InformationDoorParticipants.csv")

####ROSTER PARENT#######

dfs = []

for filename in glob.glob(os.path.join(folder_path, '*Listas*.csv')):

    print(f"Processing {filename}")
    df = pd.read_csv(filename)
    df['filename'] = filename
    pattern_to_remove = pattern + 'Listas_'
    df['filename'] = df['filename'].str.replace(pattern_to_remove, '', regex=True)
    df['filename'] = df['filename'].str.replace(r'_.*', '', regex=True)
    df['filename'] = df['filename'].astype(int)
    dfs.append(df)
    
rosters_parent = pd.concat(dfs, ignore_index=True)
rosters_parent = rosters_parent.sort_values(by='filename', ascending=False)
rosters_parent = rosters_parent.drop(columns=['filename'])
rosters_parent = rosters_parent.drop_duplicates(subset='GlobalRecordId', keep='first')

rosters_parent.to_csv("CSV_Merged/Roster_Parent.csv")

####ROSTER CHILD 1#######

dfs = []

for filename in glob.glob(os.path.join(folder_path, '*_mainhousehold*.csv')):

    print(f"Processing {filename}")
    df = pd.read_csv(filename)
    df['filename'] = filename
    pattern_to_remove = pattern + '_mainhousehold_'
    df['filename'] = df['filename'].str.replace(pattern_to_remove, '', regex=True)
    df['filename'] = df['filename'].str.replace(r'_.*', '', regex=True)
    df['filename'] = df['filename'].astype(int)
    dfs.append(df)
    
mainhousehold = pd.concat(dfs, ignore_index=True)

mainhousehold['num_missing'] = mainhousehold.isna().sum(axis=1)
mainhousehold['num_missing'] = mainhousehold['num_missing'].max() - mainhousehold['num_missing']

mainhousehold = mainhousehold.sort_values(by=['filename', 'num_missing'], ascending=[False, True])

mainhousehold = mainhousehold.drop(columns=['filename','num_missing'])
mainhousehold = mainhousehold.drop_duplicates(subset='GlobalRecordId', keep='first')

mainhousehold.to_csv("CSV_Merged/MainHousehold.csv")

####ROSTER CHILD 2#######

dfs = []

for filename in glob.glob(os.path.join(folder_path, '*_participants*.csv')):

    print(f"Processing {filename}")
    df = pd.read_csv(filename)
    df['filename'] = filename
    pattern_to_remove = pattern + '_participants_'
    df['filename'] = df['filename'].str.replace(pattern_to_remove, '', regex=True)
    df['filename'] = df['filename'].str.replace(r'_.*', '', regex=True)
    df['filename'] = df['filename'].astype(int)
    dfs.append(df)
    
participants = pd.concat(dfs, ignore_index=True)

participants['num_missing'] = participants.isna().sum(axis=1)
participants['num_missing'] = participants['num_missing'].max() - participants['num_missing']

participants = participants.sort_values(by=['filename', 'num_missing'], ascending=[False, True])

participants = participants.drop(columns=['filename','num_missing'])
participants = participants.drop_duplicates(subset='GlobalRecordId', keep='first')

participants.to_csv("CSV_Merged/Participants.csv")

####ROSTER CHILD 3#######

dfs = []

for filename in glob.glob(os.path.join(folder_path, '*_rosterhousehold*.csv')):

    print(f"Processing {filename}")
    df = pd.read_csv(filename)
    df['filename'] = filename
    pattern_to_remove = pattern + '_rosterhousehold_'
    df['filename'] = df['filename'].str.replace(pattern_to_remove, '', regex=True)
    df['filename'] = df['filename'].str.replace(r'_.*', '', regex=True)
    df['filename'] = df['filename'].astype(int)
    dfs.append(df)
    
nonparticipants = pd.concat(dfs, ignore_index=True)

nonparticipants['num_missing'] = nonparticipants.isna().sum(axis=1)
nonparticipants['num_missing'] = nonparticipants['num_missing'].max() - nonparticipants['num_missing']

nonparticipants = nonparticipants.sort_values(by=['filename', 'num_missing'], ascending=[False, True])

nonparticipants = nonparticipants.drop(columns=['filename','num_missing'])

nonparticipants = nonparticipants.drop_duplicates(subset='GlobalRecordId', keep='first')

nonparticipants.to_csv("CSV_Merged/NonParticipants.csv")

####ROSTER CHILD 4#######

dfs = [] 

for filename in glob.glob(os.path.join(folder_path, '*_nonresidentchildren*.csv')):

    print(f"Processing {filename}")
    df = pd.read_csv(filename)
    df['filename'] = filename
    pattern_to_remove = pattern + '_nonresidentchildren_'
    df['filename'] = df['filename'].str.replace(pattern_to_remove, '', regex=True)
    df['filename'] = df['filename'].str.replace(r'_.*', '', regex=True)
    df['filename'] = df['filename'].astype(int)
    dfs.append(df)
    
nonresidentchildren = pd.concat(dfs, ignore_index=True)

nonresidentchildren['num_missing'] = nonresidentchildren.isna().sum(axis=1)
nonresidentchildren['num_missing'] = nonresidentchildren['num_missing'].max() - nonresidentchildren['num_missing']

nonresidentchildren = nonresidentchildren.sort_values(by=['filename', 'num_missing'], ascending=[False, True])

nonresidentchildren = nonresidentchildren.drop(columns=['filename','num_missing'])

nonresidentchildren = nonresidentchildren.drop_duplicates(subset='GlobalRecordId', keep='first')

nonresidentchildren.to_csv("CSV_Merged/NonResidentChildren.csv")

if country != "CU":
    dfs = []  # list to hold all dataframes
    for filename in glob.glob(os.path.join(folder_path, '*Sangre*.csv')):
        print(f"Processing {filename}")
        df = pd.read_csv(filename)
        df['filename'] = filename
        pattern_to_remove = pattern + 'Sangre_'
        df['filename'] = df['filename'].str.replace(pattern_to_remove, '', regex=True)
        df['filename'] = df['filename'].str.replace(r'_.*', '', regex=True)
        df['filename'] = df['filename'].astype(int)
        dfs.append(df)
        
    sangre = pd.concat(dfs, ignore_index=True)
    sangre = sangre.sort_values(by='filename', ascending=False)
    sangre = sangre.drop(columns=['filename'])
    sangre = sangre.drop_duplicates(subset='GlobalRecordId', keep='first')
    
    sangre.to_csv("CSV_Merged/Sangre.csv")

Processing /Users/chrissoria/Google Drive/Other Computers/My Laptop (1)/documents/cadas/data/CADAS data upload/Puerto_Rico/All_CSVs/_cognitive_202406050824_0_PR08.csv
Processing /Users/chrissoria/Google Drive/Other Computers/My Laptop (1)/documents/cadas/data/CADAS data upload/Puerto_Rico/All_CSVs/_cognitive_202403221159_0_PR_3.csv
Processing /Users/chrissoria/Google Drive/Other Computers/My Laptop (1)/documents/cadas/data/CADAS data upload/Puerto_Rico/All_CSVs/_cognitive_202408080444_0_PR08.csv
Processing /Users/chrissoria/Google Drive/Other Computers/My Laptop (1)/documents/cadas/data/CADAS data upload/Puerto_Rico/All_CSVs/_cognitive_202411080148_0_PR04.csv
Processing /Users/chrissoria/Google Drive/Other Computers/My Laptop (1)/documents/cadas/data/CADAS data upload/Puerto_Rico/All_CSVs/_cognitive_202303170824_0_PR07.csv
Processing /Users/chrissoria/Google Drive/Other Computers/My Laptop (1)/documents/cadas/data/CADAS data upload/Puerto_Rico/All_CSVs/_cognitive_202408210258_0_PR06.cs

/var/folders/89/6bdxzk2j30v5n3wstywbcpg80000gn/T/ipykernel_86474/217939365.py:36: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  cog_child['num_missing'] = cog_child.isna().sum(axis=1)
/var/folders/89/6bdxzk2j30v5n3wstywbcpg80000gn/T/ipykernel_86474/217939365.py:57: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  images_df.rename(columns=rename_dict, inplace=True)
/var/folders/89/6bdxzk2j30v5n3wstywbcpg80000gn/T/ipykernel_86474/217939365.py:105: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor 

Processing /Users/chrissoria/Google Drive/Other Computers/My Laptop (1)/documents/cadas/data/CADAS data upload/Puerto_Rico/All_CSVs/Cognitiva_202404050052_0_PR03.csv
Processing /Users/chrissoria/Google Drive/Other Computers/My Laptop (1)/documents/cadas/data/CADAS data upload/Puerto_Rico/All_CSVs/Cognitiva_202411130934_0_PR10.csv
Processing /Users/chrissoria/Google Drive/Other Computers/My Laptop (1)/documents/cadas/data/CADAS data upload/Puerto_Rico/All_CSVs/Cognitiva_202411080209_0_PR06.csv
Processing /Users/chrissoria/Google Drive/Other Computers/My Laptop (1)/documents/cadas/data/CADAS data upload/Puerto_Rico/All_CSVs/Cognitiva_202407110054_0_PR04.csv
Processing /Users/chrissoria/Google Drive/Other Computers/My Laptop (1)/documents/cadas/data/CADAS data upload/Puerto_Rico/All_CSVs/Cognitiva_202112220316_0_PR08.csv
Processing /Users/chrissoria/Google Drive/Other Computers/My Laptop (1)/documents/cadas/data/CADAS data upload/Puerto_Rico/All_CSVs/Cognitiva_202408080500_0_PR04.csv
Proc

Processing /Users/chrissoria/Google Drive/Other Computers/My Laptop (1)/documents/cadas/data/CADAS data upload/Puerto_Rico/All_CSVs/_household_202411080148_0_PR04.csv
Processing /Users/chrissoria/Google Drive/Other Computers/My Laptop (1)/documents/cadas/data/CADAS data upload/Puerto_Rico/All_CSVs/_household_202408080444_0_PR08.csv
Processing /Users/chrissoria/Google Drive/Other Computers/My Laptop (1)/documents/cadas/data/CADAS data upload/Puerto_Rico/All_CSVs/_household_202403221159_0_PR_3.csv
Processing /Users/chrissoria/Google Drive/Other Computers/My Laptop (1)/documents/cadas/data/CADAS data upload/Puerto_Rico/All_CSVs/_household_202406050824_0_PR08.csv
Processing /Users/chrissoria/Google Drive/Other Computers/My Laptop (1)/documents/cadas/data/CADAS data upload/Puerto_Rico/All_CSVs/_household_202406050410_0_PR01.csv
Processing /Users/chrissoria/Google Drive/Other Computers/My Laptop (1)/documents/cadas/data/CADAS data upload/Puerto_Rico/All_CSVs/_household_202411080211_0_PR06.cs

/var/folders/89/6bdxzk2j30v5n3wstywbcpg80000gn/T/ipykernel_86474/217939365.py:186: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  household_child['num_missing'] = household_child.isna().sum(axis=1)


...

Processing /Users/chrissoria/Google Drive/Other Computers/My Laptop (1)/documents/cadas/data/CADAS data upload/Puerto_Rico/All_CSVs/Informante_202408080444_0_PR08.csv
Processing /Users/chrissoria/Google Drive/Other Computers/My Laptop (1)/documents/cadas/data/CADAS data upload/Puerto_Rico/All_CSVs/Informante_202403221159_0_PR_3.csv
Processing /Users/chrissoria/Google Drive/Other Computers/My Laptop (1)/documents/cadas/data/CADAS data upload/Puerto_Rico/All_CSVs/Informante_202303170824_0_PR07.csv
Processing /Users/chrissoria/Google Drive/Other Computers/My Laptop (1)/documents/cadas/data/CADAS data upload/Puerto_Rico/All_CSVs/Informante_202404050221_0_PR04.csv
Processing /Users/chrissoria/Google Drive/Other Computers/My Laptop (1)/documents/cadas/data/CADAS data upload/Puerto_Rico/All_CSVs/Informante_202311030641_0_91354b0c37cb46f8.csv
Processing /Users/chrissoria/Google Drive/Other Computers/My Laptop (1)/documents/cadas/data/CADAS data upload/Puerto_Rico/All_CSVs/Informante_2024032200

Processing /Users/chrissoria/Google Drive/Other Computers/My Laptop (1)/documents/cadas/data/CADAS data upload/Puerto_Rico/All_CSVs/_sociodemographic_202407110051_0_PR08 (1).csv
Processing /Users/chrissoria/Google Drive/Other Computers/My Laptop (1)/documents/cadas/data/CADAS data upload/Puerto_Rico/All_CSVs/_sociodemographic_202403220011_0_PR04.csv
Processing /Users/chrissoria/Google Drive/Other Computers/My Laptop (1)/documents/cadas/data/CADAS data upload/Puerto_Rico/All_CSVs/_sociodemographic_202404050052_0_PR03.csv
Processing /Users/chrissoria/Google Drive/Other Computers/My Laptop (1)/documents/cadas/data/CADAS data upload/Puerto_Rico/All_CSVs/_sociodemographic_202411080209_0_PR06.csv
Processing /Users/chrissoria/Google Drive/Other Computers/My Laptop (1)/documents/cadas/data/CADAS data upload/Puerto_Rico/All_CSVs/_sociodemographic_202411130934_0_PR10.csv
Processing /Users/chrissoria/Google Drive/Other Computers/My Laptop (1)/documents/cadas/data/CADAS data upload/Puerto_Rico/All

/var/folders/89/6bdxzk2j30v5n3wstywbcpg80000gn/T/ipykernel_86474/217939365.py:351: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  socio_child['num_missing'] = socio_child.isna().sum(axis=1)


Processing /Users/chrissoria/Google Drive/Other Computers/My Laptop (1)/documents/cadas/data/CADAS data upload/Puerto_Rico/All_CSVs/Sociodemográfica_202408080444_0_PR08.csv
Processing /Users/chrissoria/Google Drive/Other Computers/My Laptop (1)/documents/cadas/data/CADAS data upload/Puerto_Rico/All_CSVs/Sociodemográfica_202403221159_0_PR_3.csv
Processing /Users/chrissoria/Google Drive/Other Computers/My Laptop (1)/documents/cadas/data/CADAS data upload/Puerto_Rico/All_CSVs/Sociodemográfica_202303170824_0_PR07.csv
Processing /Users/chrissoria/Google Drive/Other Computers/My Laptop (1)/documents/cadas/data/CADAS data upload/Puerto_Rico/All_CSVs/Sociodemográfica_202411080148_0_PR04.csv
Processing /Users/chrissoria/Google Drive/Other Computers/My Laptop (1)/documents/cadas/data/CADAS data upload/Puerto_Rico/All_CSVs/Sociodemográfica_202411130934_0_PR10.csv
Processing /Users/chrissoria/Google Drive/Other Computers/My Laptop (1)/documents/cadas/data/CADAS data upload/Puerto_Rico/All_CSV

Processing /Users/chrissoria/Google Drive/Other Computers/My Laptop (1)/documents/cadas/data/CADAS data upload/Puerto_Rico/All_CSVs/Listas_202406061106_0_PR03.csv
Processing /Users/chrissoria/Google Drive/Other Computers/My Laptop (1)/documents/cadas/data/CADAS data upload/Puerto_Rico/All_CSVs/Listas_202411080147_0_PR04.csv
Processing /Users/chrissoria/Google Drive/Other Computers/My Laptop (1)/documents/cadas/data/CADAS data upload/Puerto_Rico/All_CSVs/Listas_202411070219_0_PR07.csv
Processing /Users/chrissoria/Google Drive/Other Computers/My Laptop (1)/documents/cadas/data/CADAS data upload/Puerto_Rico/All_CSVs/_mainhousehold_202411081105_0_PR07.csv
Processing /Users/chrissoria/Google Drive/Other Computers/My Laptop (1)/documents/cadas/data/CADAS data upload/Puerto_Rico/All_CSVs/_mainhousehold_202406050410_0_PR01.csv
Processing /Users/chrissoria/Google Drive/Other Computers/My Laptop (1)/documents/cadas/data/CADAS data upload/Puerto_Rico/All_CSVs/_mainhousehold_202411080211_0_PR06.cs